Este notebook está vinculado al caso de estudio "Yapo" (ver el [documento](https://docs.google.com/document/d/1SwhwMOyG7-cmzasDeisQZ_NzX5v2VoZ7--lI4UhJz_U/edit#heading=h.5qaqs0frqkx4) de presentación). Se busca construir una base de datos de anuncios en Los Rios recopilando algunos anuncios de Yapo.cl.


# 1. Construcción de la base de datos

In [2]:
import mysql.connector 

db_connection = mysql.connector.connect(user="root",host="localhost",password="root")
cursor = db_connection.cursor()

In [40]:
cursor.execute("DROP DATABASE Yapo;")

In [41]:
cursor.execute("CREATE DATABASE Yapo;")
cursor.execute("USE Yapo")

#tabla vendedor
cursor.execute("CREATE TABLE vendedor (telefono VARCHAR(20) PRIMARY KEY, "+
               "nombre VARCHAR(100), ciudad VARCHAR(30), codigo_region VARCHAR(3), nombre_region VARCHAR(30), "+
               "fecha_inscripcion VARCHAR(30))")


#tabla anuncio
cursor.execute("CREATE TABLE anuncio (url VARCHAR(300) PRIMARY KEY, "+
               "titulo VARCHAR(200), descripcion MEDIUMTEXT, precio VARCHAR(30), categoria VARCHAR(50), "
               +"telefono VARCHAR(20), FOREIGN KEY (telefono) REFERENCES vendedor(telefono))")

# 2. Scrapping de datos para llenar nuestra base de datos Yapo

In [6]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import pandas as pd
import urllib.request
from selenium import webdriver
import time

Con la librería Selenium, abriremos un navegador Chrome, en la página de los anuncios de Yapo en la región de Los Rios...

In [42]:
browser = webdriver.Chrome()
myUrl = 'https://www.yapo.cl/los_rios/todos_los_avisos?ca=11_s&l=0&w=1&cmn=243'
browser.get(myUrl)

Con la librería BeautifulSoup, realizamos un scrapping del código HTML para recuperar el enlace de la "última página"...

In [43]:
pageSoup = soup(browser.page_source, 'html.parser')

pages = pageSoup.find('span',  {'class', 'nohistory FloatRight'}).a['href']

In [44]:
pages

'https://www.yapo.cl/los_rios?ca=11_s&cmn=243&o=655'

El parametro 'o' de la URL corresponde al número de la pagina en la lista de anuncios.

In [45]:
index = pages.rfind('=')
print(index)
lastPage = int(pages[index+1:])
print(lastPage)
root_pages = pages[:index+1]
print(root_pages)

46
655
https://www.yapo.cl/los_rios?ca=11_s&cmn=243&o=


- la variable "index" permite identificar el indice del último caracter '='.

- la variable "lastPage" permite identificar el numero de la última página en la URL.

- la variable "root_pages" permite aislar la cadena de caracteres que corresponde a la raiz de la URL (sin el numero de página).


Empezaremos la iteración sobre cada página que escrapear...

In [ ]:
for i in range(lastPage):
    
    #recuperarmos la URL de la pagina corriente
    url = root_pages + str(i+1)
    
    #con Selenium, vamos en navegar en esta página
    browser.get(url)
    
    #empezamos el scrapping de la pagina corriente
    pageSoup = soup(browser.page_source, 'html.parser')
    
    #recuperamos todos los tags HTML que corresponden a la lista de anuncios en esta pagina
    links = pageSoup.findAll('td', {'class' : 'thumbs_subject'})
    
    
    #empezamos a iterar sobre cada anuncio
    for link in links:
        
        #todos los datos que necesitamos encontrar
        url, titulo, descripcion, precio, categoria="","","","",""
        telefono, nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion="","","","","",""
        
        #Navegamos hacia la pagina del anuncio
        url=link.find('a',{'class':'title'})['href']
        print(url)
        browser.get(link.find('a',{'class':'title'})['href'])
        
        
        #RECUPERAMOS EL TITULO DEL ANUNCIO           
        pageSoup = soup(browser.page_source, 'html.parser')
        if(pageSoup.find('h1', {"id" : "da_subject"})):
            titulo = pageSoup.find('h1', {"id" : "da_subject"}).text.strip()
            print(titulo)
            
        #RECUPERAMOS LA DESCRIPCION DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "description"})):
            try:
                descripcion = pageSoup.find('div', {"class" : "description"}).text.split(' ', 1)[1].strip().replace(u'\n', u' ')
            except:
                continue
                
        #RECUPERAMOS EL PRECIO DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "price text-right"})):
            precio = pageSoup.find('div', {"class" : "price text-right"}).text.strip().replace(u'\n', u' ').replace(u'\t', u'')
            print("precio:"+precio)
            
        #RECUPERAMOS LA CATEGORIA DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "breadcrumbs"})):
            categoria = pageSoup.find('div', {"class" : "breadcrumbs"}).find('a', {"id" : "breadcrumb_category"}).find('strong').text.strip().replace(u'\n', u' ')
            print(categoria)
            
        #RECUPERAMOS EL TELEFONO DEL VENDEDOR
        
        #...
        
        #RECUPERAMOS EL NOMBRE DEL VENDEDOR
        if(pageSoup.find('aside', {"class" : "sidebar-right"})):
            aside = pageSoup.find('aside', {"class" : "sidebar-right"})
            nombre=aside.find('seller-info').attrs['username']
            
        #RECUPERAMOS LA CIUDAD DEL VENDEDOR
        
        #...
        
        #RECUPERAMOS EL CODIGO Y NOMBRE DE LA REGION
        
        #...
        
        #RECUPERAMOS LA FECHA DE INSCRIPCION
        
        #...
        
        #...
        
        # LLENAMOS LA BASE DE DATOS: TABLA VENDEDOR
        
        #...
        
        # LLENAMOS LA BASE DE DATOS: TABLA ANUNCIO
        try:
            sql = "INSERT INTO anuncio (url, titulo, descripcion, precio, categoria) VALUES (%s, %s, %s,%s,%s)"
            val = (url, titulo, descripcion, precio, categoria)
            cursor.execute(sql, val)
        except:
            continue
        
        cursor.execute("COMMIT")

https://www.yapo.cl/los_rios/television_camaras/audifonos_bluetooth_73780710.htm?ca=11_s&first=1&oa=73780710&xsp=0
Audífonos Bluetooth
precio:$ 19.990
Audio, TV, video y fotografía
https://www.yapo.cl/los_rios/consolas_videojuegos/crysis_2__ps3___73780658.htm?ca=11_s&oa=73780658&xsp=1
Crysis 2. ps3.
precio:$ 10.000
Consolas, videojuegos y accesorios
https://www.yapo.cl/los_rios/celulares/lote_carcasas_iphone_4_4s_73780570.htm?ca=11_s&oa=73780570&xsp=2
Lote carcasas iphone 4/4s
precio:$ 10.000
Celulares, teléfonos y accesorios
https://www.yapo.cl/los_rios/camiones_furgones/camion_ford_cargo_1517_73780556.htm?ca=11_s&oa=73780556&xsp=3
Camion Ford Cargo 1517
precio:$ 7.000.000
Buses, camiones y furgones
https://www.yapo.cl/los_rios/autos/toyota_yaris_2006_73780540.htm?ca=11_s&oa=73780540&xsp=4
Toyota yaris 2006
precio:$ 3.250.000
Autos, camionetas y 4x4
https://www.yapo.cl/los_rios/celulares/popsocket_73780526.htm?ca=11_s&oa=73780526&xsp=5
Popsocket
precio:$ 2.000
Celulares, teléfonos y a

Bolsas ecológicas lavables
precio:$ 1.000
Coches y artículos infantiles
https://www.yapo.cl/los_rios/arrendar/busco_parcela__73778557.htm?ca=11_s&oa=73778557&xsp=48
Busco Parcela
precio:$ 4.000.000
Arriendo - Terreno
https://www.yapo.cl/los_rios/calzado/taco_fiesta_con_brillo_73778549.htm?ca=11_s&oa=73778549&xsp=49
Taco fiesta con brillo
precio:$ 10.000
Calzado
https://www.yapo.cl/los_rios/arrendar/busco_parcela__73778557.htm?ca=11_s&oa=73778557&xsp=0
Busco Parcela
precio:$ 4.000.000
Arriendo - Terreno
https://www.yapo.cl/los_rios/calzado/taco_fiesta_con_brillo_73778549.htm?ca=11_s&oa=73778549&xsp=1


# 3. Consultas SQL

- ¿Cuál es el precio promedio de los anuncios por categoría?

- ¿Cuál es el número de anuncios por vendedor y por día?

- ¿Insertar una columna “Perfil” en la tabla Vendedor. Los Vendedores que publicaron más de 5 anuncios en los últimos 7 días, tienen un perfil “Pro”, los otros se consideran “Personal”.

- ¿Cuál es el precio promedio de los anuncios según el día de la semana (lunes, martes, miercoles, etc.)


In [ ]:
#¿Cuál es el precio promedio de los anuncios por categoría?

